In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

In [ ]:
#read the data file from google sheet
def read_google_sheet(file , sheet_name ) :
    scope = ["https://spreadsheets.google.com/feeds",
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('sheet_secret.json', scope)
    client = gspread.authorize(creds)
    spreadsheet = client.open(file)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    return df

In [ ]:
data = read_google_sheet('analyse' , 'Feuille 1')
print(data)

   age  revenu genre          promotion    reduction          acharnement  \
0   21     100     H  reduction_directe      plus_75  tout_a_fait_daccord   
1   22     100     H  reduction_directe  entre_51_75  pas_du_tout_daccord   
2   20     100     H              bogof     moins_10  tout_a_fait_daccord   
3   12    1000     H      offre_groupee  entre_10_25       plutot_daccord   
4   20    3000     F  reduction_directe  entre_51_75  pas_du_tout_daccord   
5   22       0     H            gratuit  entre_51_75  tout_a_fait_daccord   
6   25    1000     H  reduction_directe  entre_10_25       plutot_daccord   
7   22    5000     F  reduction_directe  entre_26_50       plutot_daccord   
8   18       0     F  reduction_directe      plus_75       plutot_daccord   

              facteur              attitude                   produit  \
0  avis_consommateurs         fidele_marque               accessoires   
1   utilite_immediate  recherche_promotions  smartphone_haut_de_gamme   
2       du

In [ ]:
def most_liked_prom(x):
       freq = x.value_counts()
       return pd.Series({
            'promotion_la_plus_frequente': freq.idxmax(),
            'nombre_occurrences': freq.max()
        })

df_promo_product = (
    data.groupby('produit')['promotion']
    .agg([
        ('promotion_la_plus_frequente', lambda x: x.value_counts().idxmax()),
        ('nombre_occurrences', lambda x: x.value_counts().max())
    ])
    .reset_index()
)

df_promo_product.columns = ['produit', 'promotion_la_plus_frequente', 'nombre_occurrences']

print(df_promo_product)

                    produit promotion_la_plus_frequente  nombre_occurrences
0               accessoires           reduction_directe                   2
1          objets_connectes               offre_groupee                   1
2     ordinateurs_portables           reduction_directe                   2
3  smartphone_haut_de_gamme           reduction_directe                   2


In [ ]:
from utils import write_xl_file
from utils import clear_google_sheet

In [ ]:
clear_google_sheet("promo_product" , "Feuille 1")

Aucune ligne à supprimer en dehors de l’en-tête.


In [ ]:
write_xl_file("promo_product", df_promo_product)

Les données ont été téléchargées dans Google Sheets.


In [ ]:
min_income = data['revenu'].min()
max_income = data['revenu'].max()
print(max_income)

5000


In [ ]:
bins = [min_income , (min_income + max_income) / 3 , 2 * (min_income + max_income) / 3, max_income]
labels_intervals = ['bas' , 'moyen' , 'eleve']
data['revenu_intervalle'] = pd.cut(data['revenu'], bins=bins, labels=labels_intervals, include_lowest=True)
print(data)

   age  revenu genre          promotion    reduction          acharnement  \
0   21     100     H  reduction_directe      plus_75  tout_a_fait_daccord   
1   22     100     H  reduction_directe  entre_51_75  pas_du_tout_daccord   
2   20     100     H              bogof     moins_10  tout_a_fait_daccord   
3   12    1000     H      offre_groupee  entre_10_25       plutot_daccord   
4   20    3000     F  reduction_directe  entre_51_75  pas_du_tout_daccord   
5   22       0     H            gratuit  entre_51_75  tout_a_fait_daccord   
6   25    1000     H  reduction_directe  entre_10_25       plutot_daccord   
7   22    5000     F  reduction_directe  entre_26_50       plutot_daccord   
8   18       0     F  reduction_directe      plus_75       plutot_daccord   

              facteur              attitude                   produit  \
0  avis_consommateurs         fidele_marque               accessoires   
1   utilite_immediate  recherche_promotions  smartphone_haut_de_gamme   
2       du

In [ ]:
def prom_for_each_income(df , filter_type) :
    promo = df [df['revenu_intervalle'] == filter_type]['promotion']
    values = [[val] for val in promo.tolist()]
    df_filtered = pd.DataFrame(values , columns = ['promotion'] )
    return df_filtered

In [ ]:
#print(prom_for_each_income(data , 'bas'))
print(prom_for_each_income(data , 'moyen'))
print(prom_for_each_income(data , 'eleve'))


           promotion
0  reduction_directe
           promotion
0  reduction_directe


In [ ]:
clear_google_sheet("bas" , "Feuille 1")
write_xl_file("bas", prom_for_each_income(data , 'bas'))

6 lignes supprimées (l’en-tête a été conservé).
Les données ont été téléchargées dans Google Sheets.


In [ ]:
clear_google_sheet("moyen" , "Feuille 1")
write_xl_file("moyen", prom_for_each_income(data , 'moyen'))

1 lignes supprimées (l’en-tête a été conservé).
Les données ont été téléchargées dans Google Sheets.


In [ ]:
clear_google_sheet("eleve" , "Feuille 1")
write_xl_file("eleve", prom_for_each_income(data , 'eleve'))

1 lignes supprimées (l’en-tête a été conservé).
Les données ont été téléchargées dans Google Sheets.
